In [1]:
import os

In [2]:
os.chdir("./../.")

In [3]:
import dotenv

In [ ]:
dotenv.load_dotenv()

In [5]:
import os
from tqdm import tqdm
from datetime import datetime, timedelta, timezone
from src.aws_utils import send_message_to_queue
from src.dependencies import get_db

In [6]:
INTERVAL_MINUTES = int(os.getenv("INTERVAL", "5"))


def main():
    left_datetime = datetime.now(
        timezone.utc)

    right_datetime = left_datetime + timedelta(seconds=INTERVAL_MINUTES*60)

    db = get_db()

    subscriptions = list(db['NewsSubscription'].find({
        '$and': [{
            'status': {'$ne': 'RUNNING'},
            'nextRunTime': {
                '$gte': left_datetime,
                '$lte': right_datetime
            }
        }]
    }))

    for sub in tqdm(subscriptions):
        subscriptionId = sub["_id"]
        send_message_to_queue(subscriptionId)
        db['NewsSubscription'].update_one(
            filter={"_id": subscriptionId},
            update={"$set": {"status": "RUNNING"}}
        )

In [ ]:
main()